# Load libraries

# Load model

# Calculate zero-flux reactions across conditions

In [ ]:
def calculate_zero_flux(cobra_model, optimal_percentages=[], tol=1e-6):
    zero_flux_all_counts = []
    for percentage in optimal_percentages:
        print(percentage)
    
        fva = cobra.flux_analysis.flux_variability_analysis(roseburia_cobra_model, fraction_of_optimum=percentage)

        zero_flux = fva.loc[ (abs(fva['minimum']) < tol ) & (abs(fva['maximum']) < tol)]
        zero_flux_reactions = len(zero_flux.index.tolist())
        
        zero_flux_all_counts.append(zero_flux_reactions)
    
    return zero_flux_all_counts
        

optimal_percentages=[0, 0.99, 1-1e-9, 1-1e-10, 1]
zero_flux_all_counts = calculate_zero_flux(roseburia_cobra_model, optimal_percentages=optimal_percentages, tol=1e-6)

print(zero_flux_all_counts)


# Plot the number of reactions with zero flux across conditions

In [ ]:
optimal_percentages_str =['0', '0.99', '1-1e-9', '1-1e-10', '1']

plt.plot(optimal_percentages_str, zero_flux_all_counts, marker='o', linestyle='-', color='b')
plt.xlabel("Optimal Percentage (scaled from 0 to 1)")
plt.ylabel("Zero Flux Reactions")
#plt.title("Line Plot")

plt.legend()
plt.show()


# Create boolean arrays based on active reactions

In [ ]:
def calculate_zero_flux(cobra_model, optimal_percentages=[], tol=1e-6, reactions_list=[]):
    zero_flux_bools_all = []
    for percentage in optimal_percentages:
        print(percentage)
    
        fva = cobra.flux_analysis.flux_variability_analysis(roseburia_cobra_model, fraction_of_optimum=percentage)

        zero_flux_bool = np.where((abs(fva['minimum']) < tol) & (abs(fva['maximum']) < tol), 0, 1)
        print(zero_flux_bool)
        
        zero_flux_bools_all.append(zero_flux_bool)
    
    return zero_flux_bools_all
        

optimal_percentages=[0, 0.99, 1-1e-9, 1-1e-10, 1]
boolean_arrays = calculate_zero_flux(roseburia_cobra_model, optimal_percentages=optimal_percentages, tol=1e-6, reactions_list=reactions)


# Calculate hamming distance and Plot heatmap

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

def plot_hamming_heatmap(boolean_arrays, optimal_percentages):
    """
    Compute pairwise Hamming distances and plot a heatmap.
    
    Parameters:
    boolean_arrays (list of np.arrays): List of binary arrays (0 = zero flux, 1 = nonzero).
    optimal_percentages (list): The corresponding percentages used in the analysis.
    """
    # Convert list of boolean arrays to a 2D NumPy array
    binary_matrix = np.array(boolean_arrays)
    
    # Compute pairwise Hamming distances
    hamming_distances = pdist(binary_matrix, metric="hamming")
    
    # Convert to square matrix
    hamming_matrix = squareform(hamming_distances)
    
    # Create a heatmap
    #plt.figure(figsize=(8, 6))
    sns.clustermap(hamming_matrix, cbar=False, cbar_pos=None, annot=True, fmt=".2f", cmap="coolwarm", xticklabels=optimal_percentages, yticklabels=optimal_percentages, method="ward")
    #sns.heatmap(hamming_matrix, annot=True, fmt=".2f", cmap="coolwarm", xticklabels=optimal_percentages, yticklabels=optimal_percentages)
    
    #plt.title("Pairwise Hamming Distance Heatmap")
    #plt.xlabel("Optimal Percentages")
    #plt.ylabel("Optimal Percentages")
    plt.show()


plot_hamming_heatmap(boolean_arrays, optimal_percentages)

